<a href="https://colab.research.google.com/github/odeepak/Stock_Market_Performance_Analysis/blob/main/Stock_Market_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stock Market Performance Analysis


In [1]:
#upload csv file
from google.colab import files


uploaded = files.upload()


Saving stocks.csv to stocks.csv


In [2]:
#install yfinance
!pip install yfinance

In [4]:
#import lib
#Now below is how we can collect real-time stock market data using the yfinance API:
#we first imported the necessary Python libraries and downloaded the historical stock price data for four companies: Apple, Microsoft, Netflix, and Google, for the last three months.
import pandas as pd
import yfinance as yf
from datetime import datetime
start_date = datetime.now() - pd.DateOffset(months=3)
end_date = datetime.now()

tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

df_list = []

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date)
    df_list.append(data)

df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date'])
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-06-27  185.889999  188.389999  185.669998  188.059998  187.806381   
       2023-06-28  187.929993  189.899994  187.600006  189.250000  188.994781   
       2023-06-29  189.080002  190.070007  188.940002  189.589996  189.334320   
       2023-06-30  191.630005  194.479996  191.259995  193.970001  193.708420   
       2023-07-03  193.779999  193.880005  191.759995  192.460007  192.200470   

                     Volume  
Ticker Date                  
AAPL   2023-06-27  50730800  
       2023-06-28  51216800  
       2023-06-29  46347300  
      

In [5]:
df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-06-27  185.889999  188.389999  185.669998  188.059998   
1   AAPL 2023-06-28  187.929993  189.899994  187.600006  189.250000   
2   AAPL 2023-06-29  189.080002  190.070007  188.940002  189.589996   
3   AAPL 2023-06-30  191.630005  194.479996  191.259995  193.970001   
4   AAPL 2023-07-03  193.779999  193.880005  191.759995  192.460007   

    Adj Close    Volume  
0  187.806381  50730800  
1  188.994781  51216800  
2  189.334320  46347300  
3  193.708420  85069600  
4  192.200470  31458200  


In [6]:
import plotly.express as px
fig = px.line(df, x='Date',
              y='Close',
              color='Ticker',
              title="Stock Market Performance for the Last 3 Months")
fig.show()

In [7]:
#Now let’s look at the faceted area chart, which makes it easy to compare the performance of different companies and identify similarities or differences in their stock price movements:
fig = px.area(df, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
fig.show()

In [8]:
#Now let’s analyze moving averages, which provide a useful way to identify trends and patterns in each company’s stock price movements over a period of time:
df['MA10'] = df.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
df['MA20'] = df.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
          MA10      MA20
0          NaN       NaN
1          NaN       NaN
2          NaN       NaN
3          NaN       NaN
4          NaN       NaN
..         ...       ...
59  176.889001  180.3450
60  176.526001  179.9855
61  176.187001  179.9060
62  175.859001  179.7795
63  175.425002  179.3680

[64 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
192         NaN         NaN
193         NaN         NaN
194         NaN         NaN
195         NaN         NaN
196         NaN         NaN
..          ...         ...
251  137.438002  135.957001
252  136.954002  135.864501
253  136.359003  135.906001
254  135.802002  135.980001
255  135.140001  135.863001

[64 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
64          NaN         NaN
65          NaN         NaN
66          NaN         NaN
67          NaN         NaN
68          NaN         NaN
..          ...         ...
123  331.734998  329.552501
124  330.696997  32

In [9]:
#Now here’s how to visualize the moving averages of all companies:
for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'],
                  title=f"{ticker} Moving Averages")
    fig.show()

In [10]:
df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
fig = px.line(df, x='Date', y='Volatility',
              color='Ticker',
              title='Volatility of All Companies')
fig.show()

In [11]:
# create a DataFrame with the stock prices of Apple and Microsoft
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close': 'AAPL'})
microsoft = df.loc[df['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
df_corr = pd.merge(apple, microsoft, on='Date')

# create a scatter plot to visualize the correlation
fig = px.scatter(df_corr, x='AAPL', y='MSFT',
                 trendline='ols',
                 title='Correlation between Apple and Microsoft')
fig.show()

There is a strong linear relationship between the stock prices of Apple and Microsoft, which means that when the stock price of Apple increases, the stock price of Microsoft also tends to increase. It is a sign of a strong correlation or similarity between the two companies, which can be due to factors such as industry trends, market conditions, or common business partners or customers. For investors, this positive correlation may indicate an opportunity to diversify their portfolio by investing in both companies, as both stocks may offer similar potential returns and risks.

**Summary**
Stock Market Performance Analysis involves calculating moving averages, measuring volatility, conducting correlation analysis and analyzing various aspects of the stock market to gain a deeper understanding of the factors that affect stock prices and the relationships between the stock prices of different companies.
